In [49]:
%pip install tiktoken openai pinecone-client pandas supabase pymongo -q 

Note: you may need to restart the kernel to use updated packages.


In [30]:
import openai
import tiktoken
import pinecone
import os
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()


openai.api_key = os.getenv("OPENAI_API_KEY")

PINECONE_API_KEY = 'YOUR-PINECONE-KEY'
PINECONE_API_ENV = 'YOUR-PINECONE-ENV'

In [3]:
data = []
with open("./dump.csv","r") as file:
    data = file.read()
data = data.split("Start")
data = data[1:]

In [13]:
articles = []
for line in data:
    line = line.split("\n")
    body_mode = False
    body = ""
    article = {}
    for i in line:
        if ('本文：' in i):
            body_mode = True
            continue
        if ('TextEnd' in i):
            body_mode = False
            article["本文"] = body
        if (body_mode):
            body += i + '\n'
        else:
            if ('：' in i):
                i = i.split('：')
                article[i[0]] = i[1]
    articles.append(article)
articles

[{'掲載日': '2023/04/05(水)',
  '媒体名': '日本農業新聞',
  '面名': '総合１面１２版遅',
  '面数／ページ': '001',
  '版数': '',
  '著作権': '無日本農業新聞',
  '図表有無': '無',
  '写真有無': '有',
  '人権抵触': '無',
  '本文文字数': '966',
  '見出し': '［肥料自給\u3000これで強化］（１）\u3000堆肥入りペレット\u3000まきやすく広域流通も',
  '本文': '埼玉県加須市で水稲など５７ヘクタールを栽培する小倉祐一さん（４２）は３月、肥料成分や有機物を水田に供給するペレット状の資材を初めてまいた。資材の原料の約７割は牛ふん堆肥を中心とした国内の資源だ。小倉さんは「ブロードキャスターが使えて省力的。臭いも気にならない」という。\n農水省によると、２月の肥料価格は２０２０年を１００とした指数で１５５で、前年同月を４割上回る。輸入に大きく依存する肥料原料のリン安や塩化カリの高騰が主因。こうした中で国内資源の家畜ふん堆肥への注目が高まっている。\n小倉さんはリンやカリの成分量を抑えた安価なＬ型肥料への切り替えで、肥料費の圧縮を目指す。一方、減らした分のリン、カリに加え、ケイ酸、地力向上につながる有機物の補給へ、採用したペレット資材が「稲サポ」だ。１袋（１５キロ）当たりの肥料成分比はリン酸７％、カリ６％、ケイ酸８％などで、１０アール当たり５０キロを計４ヘクタールに散布した。\n同省によると、家畜排せつ物は年間約８０００万トン（２０２０年）発生し、８６％が主に堆肥として活用されている。一方、稲作地帯は畜産農家が少ないなどで水田への堆肥の施用量は１５年までの３０年間で４分の１に減った。\n小倉さんも、牛ふん堆肥の施用を考えたが、近隣に入手可能な農場がなかったことや散布機を持っていないこと、臭いへの懸念などから諦めてきた。稲サポのように、ペレット加工して体積を減らした資材は、堆肥では難しかった広域での流通や、従来の散布機の活用も可能になる。\n稲サポを製造する朝日アグリアは、牛ふん堆肥を群馬や埼玉県の畜産農家から仕入れる。２１年に開発して以降、埼玉や茨城、千葉といった関東圏に加え、新潟などでも利用が広がる。\n・地力向上し収量安定に\n家畜ふん堆

In [18]:
df = pd.DataFrame(articles)
df['text'] = df.見出し + " " + (df.本文).replace("\n", " ")

df

,掲載日,媒体名,面名,面数／ページ,版数,著作権,図表有無,写真有無,人権抵触,本文文字数,...,部署名,最終更新者,入電日,キーワード,発行地域,公開可否,外販期限,二次利用可否,二次利用期限,text
0,2023/04/05(水),日本農業新聞,総合１面１２版遅,001,,無日本農業新聞,無,有,無,966,...,,編集003,//,,,可,//,否,//,［肥料自給 これで強化］（１） 堆肥入りペレット まきやすく広域流通も 埼玉県加須市で水稲な...
1,2023/04/05(水),日本農業新聞,総合１面１２版遅,001,,無日本農業新聞,無,無,無,501,...,,編集003,//,,,可,//,否,//,ブロッコリー４割高 前進入荷の反動で急減 ブロッコリーの相場が急伸し、平年比４割高を付けてい...
2,2023/04/05(水),日本農業新聞,総合１面１２版遅,001,,無日本農業新聞,無,有,無,391,...,,編集003,//,,,可,//,否,//,「糸勝百合」ＢＭＳ１０・８ 岡山県種雄牛 全国歴代２位 岡山県は、黒毛和種の県基幹種雄牛に新...
3,2023/04/05(水),日本農業新聞,総合１面１２版遅,001,,無日本農業新聞,無,無,無,382,...,,編集003,//,,,可,//,否,//,クラファンで米作り体験 全中、「国消国産」ファンづくりへ ＪＡ全中は４日、クラウンドファンデ...
4,2023/04/05(水),日本農業新聞,総合１面１２版遅,001,,無日本農業新聞,無,無,無,517,...,,編集003,//,,,可,//,否,//,［四季］ ２０２３／４／５ 「お主も悪よのぉ」「お代官さまこそ」。「水戸黄門」などの時代劇に...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,2023/04/05(水),日本農業新聞,社会１２版遅,011,,無日本農業新聞,無,有,無,328,...,,編集003,//,,,可,//,否,//,ＪＡの将来像考えたい 全中副会長 菅野氏 福島大大学院に入学 ＪＡ全中の菅野孝志副会長が４日...
234,2023/04/05(水),日本農業新聞,社会１２版遅,011,,有日本農業新聞,無,無,無,327,...,,編集003,//,,,可,//,否,//,はしか、風疹接種率が低下 日本感染症学会などが加盟する予防接種推進専門協議会は、はしか、風疹...
235,2023/04/05(水),日本農業新聞,社会１２版遅,011,,有日本農業新聞,無,無,無,170,...,,編集003,//,,,可,//,否,//,Ｇ７警備徹底を 警察庁長官 ５月に迫った先進７カ国首脳会議（Ｇ７広島サミット）の大規模警備に...
236,2023/04/05(水),日本農業新聞,社会１２版遅,011,,無日本農業新聞,無,有,無,424,...,,編集003,//,,,可,//,否,//,［女の階段］ 教わった行事食 次代に 春の訪れが近くなり、心が躍るようにうれしくなります。嫁...


In [20]:
enc = tiktoken.encoding_for_model("gpt-4")
df['n_tokens'] = df.text.apply(lambda x: len(enc.encode(x)))

In [21]:
df.n_tokens.head()

0    1323
1     602
2     688
3     482
4     650
Name: n_tokens, dtype: int64

In [24]:
embeddings = OpenAIEmbeddings()
df['embeddings'] = df.text.apply(lambda x: embeddings.embed_query(x))

In [25]:
df['embeddings'].head()

0    [-0.0018489982467144728, 0.00899107288569212, ...
1    [-0.029560010880231857, -0.019771013408899307,...
2    [-0.029585866257548332, -0.00097668229136616, ...
3    [-0.010900684632360935, -0.0031318876426666975...
4    [-0.012666326947510242, -0.02228233963251114, ...
Name: embeddings, dtype: object

In [32]:
# Add an 'id' column to the DataFrame
from uuid import uuid4
df['id'] = [str(uuid4()) for _ in range(len(df))]

In [33]:
# Define index name
index_name = 'agrinews'

# Initialize connection to Pinecone
pinecone.init(api_key=os.getenv('PINECONE_API_KEY'), environment=os.getenv('PINECONE_API_ENV'))

# Check if index already exists, create it if it doesn't
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536, metric='cosine')

# Connect to the index and view index stats
index = pinecone.Index(index_name)
index.describe_index_stats()

from tqdm.auto import tqdm

batch_size = 100  # how many embeddings we create and insert at once

# Convert the DataFrame to a list of dictionaries
chunks = df.to_dict(orient='records')

# Upsert embeddings into Pinecone in batches of 100
for i in tqdm(range(0, len(chunks), batch_size)):
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    ids_batch = [x['id'] for x in meta_batch]
    embeds = [x['embeddings'] for x in meta_batch]
    meta_batch = [{
        'title': x['見出し'],
        'text': x['本文']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    index.upsert(vectors=to_upsert)

100%|██████████| 3/3 [00:10<00:00,  3.51s/it]


In [34]:
df.to_csv("agrinews.csv", index=False)

In [46]:
user_input = "Fukushima"
embed_query = embeddings.embed_query(user_input)
print(embed_query)

# retrieve from Pinecone
query_embeds = embed_query

# get relevant contexts (including the questions)
response = index.query(query_embeds, top_k=5, include_metadata=True)
response

[-0.015853596851229668, -0.008077100850641727, 0.003868643892928958, -0.006900824140757322, -0.04553498327732086, 0.0013584363041445613, -0.022192422300577164, 0.003443877212703228, -0.004064689856022596, -0.010024492628872395, -0.005352059379220009, 0.011135420762002468, 0.002082990249618888, 0.007155683822929859, -0.01512169186025858, -0.004038550425320864, 0.032909613102674484, -0.009292586706578732, 0.01702987402677536, -0.022702142596244812, 0.00335238897241652, 0.02536837011575699, -0.008599890395998955, -0.018977265805006027, -0.0016190143069252372, -0.015958156436681747, 0.00937100499868393, -0.02846589870750904, -0.006021883804351091, -0.017800988629460335, 0.02087237872183323, -0.01357946265488863, -0.01033816672861576, -0.02502855658531189, 0.007207963149994612, -0.006632894277572632, 0.0056690010242164135, -0.006698242854326963, 0.011338002048432827, -0.02815222553908825, 0.0036007140297442675, 0.0052703735418617725, -0.0018346650758758187, -0.01438978686928749, -0.00675705

{'matches': [{'id': 'aa46481f-7480-4b9b-9168-39d5e550fc39',
              'metadata': {'text': '【東京あおば】ＪＡ東京あおばは３月中旬、大規模災害に備えた事業継続計画（ＢＣＰ）の一環で災害訓練を行った。所属長が同ＪＡ本店内に設置された災害対策本部に各部署の被災状況などを報告。役職員は個人の安否情報を専用ウェブサイトのフォームに入力し、安否確認をした。\n'
                                   'ＪＡの相原和彦副組合長は「いつ起きてもおかしくない緊急事態に対応できるよう、普段から災害時の対応を心がけていきたい」と話した。\n'
                                   '\n'
                                   '\n',
                           'title': '事業継続へ災害訓練\u3000ＪＡ東京あおば'},
              'score': 0.8116135,
              'values': []},
             {'id': '7e9dc5da-94b6-4c75-a3c9-df535dcd4708',
              'metadata': {'text': '政府は４日、脱炭素化を進めるため、水素基本戦略を５月末をめどに改定する方針を明らかにした。改定案では２０４０年の水素の供給量目標について、足元の２００万トンから６倍となる１２００万トン程度まで増やすことを検討すると明記。今後１５年間で官民合わせて１５兆円を投資する計画で、二酸化炭素（ＣＯ２）の排出を抑えられる水素エネルギーの普及を後押しする。\n'
                                   '４日午前に開かれた再生可能エネルギーの普及拡大に関する会合で岸田文雄首相が改定を表明した。\n'
                                   '\n'
                                   '\n',
                        